In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE121291"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE121291"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE121291.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE121291.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE121291.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Define the variables for gene expression and trait availability
is_gene_available = True  # The dataset contains mRNA microarray data
trait_row = 2  # The experimental agent is recorded in row 2
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# Define conversion functions
def convert_trait(value):
    """Convert trait value to categorical based on treatment agent."""
    if not isinstance(value, str) or ':' not in value:
        return None
    value = value.split(':', 1)[1].strip().upper()
    # Map different treatments to numeric values
    if 'DMSO' in value:
        return 0  # Control
    elif 'SY-1365' in value:
        return 1  # Treatment of interest
    elif 'JQ1' in value:
        return 2  # Comparison treatment
    elif 'NVP2' in value:
        return 3  # Comparison treatment
    elif 'FLAVO' in value:
        return 4  # Comparison treatment
    return None

def convert_age(value):
    """Convert age to continuous value."""
    # Not implemented since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    # Not implemented since gender data is not available
    return None

# Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical feature extraction
if trait_row is not None:
    # Get sample characteristics from the previous step
    sample_characteristics = {
        0: ['disease state: Acute Myeloid Leukemia'], 
        1: ['cell line: AML cell line  THP-1'], 
        2: ['agent: DMSO', 'agent: SY-1365', 'agent: JQ1', 'agent: NVP2', 'agent: FLAVO'], 
        3: ['time: 2 hours', 'time: 6 hours']
    }
    
    # Create a DataFrame from the sample characteristics
    rows = []
    for row_idx, values in sample_characteristics.items():
        for value in values:
            rows.append({"row": row_idx, "value": value})
    clinical_data = pd.DataFrame(rows)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:", preview)
    
    # Save clinical data to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These don't appear to be standard human gene symbols. They look like probe IDs from a microarray platform,
# most likely Affymetrix (based on the "_at" suffix pattern).
# These identifiers need to be mapped to human gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# Based on the preview, 'ID' column contains the probe identifiers (like '11715100_at')
# and 'Gene Symbol' column contains the human gene symbols (like 'HIST1H3G')

# 2. Use get_gene_mapping function to extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships between probes and genes as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the gene expression data after mapping
print("Gene expression data after mapping (first 5 rows):")
print(gene_data.head())

# Check the shape of the resulting gene expression data
print(f"Shape of gene expression data: {gene_data.shape}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Clinical data loaded from {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# Create a proper clinical dataframe for linking with gene data
# We need to manually create this since our clinical data structure is not compatible with the expected format
sample_ids = normalized_gene_data.columns
# Create samples with treatment type 0 (DMSO/control) as per our trait definition
clinical_matrix = pd.DataFrame({
    'Acute_Myeloid_Leukemia': [0] * len(sample_ids)  # All assigned as control (DMSO)
}, index=sample_ids)

# 3. Link the clinical and genetic data
linked_data = clinical_matrix.join(normalized_gene_data.T)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, 'Acute_Myeloid_Leukemia')
print(f"Data shape after handling missing values: {linked_data.shape}")

# Verify that the trait column has at least two unique values
unique_trait_values = linked_data['Acute_Myeloid_Leukemia'].unique()
print(f"Unique values in trait column: {unique_trait_values}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, 'Acute_Myeloid_Leukemia')

# 6. Conduct quality check and save the cohort information
note = "Dataset contains only AML (Acute Myeloid Leukemia) samples with treatment set as control (DMSO). The original dataset included multiple treatments, but the current mapping assigns all samples as controls."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")